# AlexNet: ImageNet Classification with Deep Convolutional Neural Networks

----
----

## 1. Introduction

---

## 2. Dataset

### ImageNet
15 million labeled images with 22,000 categories

본 paper가 실린 ILSVRC에서는 ImageNet의 일부인 1.2million개의 training image와  50,000개의 validaiton, 150,000개의 testing images가 사용되었다.

### Image Preprocessing
ImageNet의 image들은 resolution이 fixed되지 않았다. 하지만 본 paper의 model은 input dimensionality가 고정되어야 한다. 따라서 몇가지 전처리 기준이 필요하다.
* sqaure image: 256 x 256의 fixed resolution으로 **down-sampling**
* rectangular image: shorter side: to 256, longer side: select 256 pixels in **center**

---

## 3. The Architecture

### 3.1. ReLU Nonlinearity
(이 paper 전까지는 거의 ``tanh``을 activation function으로 사용했었다)

$f(x)= max(0, x)$의 꼴을 가지는 ``ReLU`` function을 사용하여 학습의 속도를 비약적으로 높였다.

### 3.2 Training on Multiple GPUs
(이 paper가 나올 당시의 GPU 성능은 지금에 비해 많이 부족했으므로, 지금 이를 구현할 때에 굳이 이 파트를 신경 쓸 필요는 없을 듯)

GPU 1개로 이 model을 설계하기에는 부족하므로, 2개의 GPU을 cross-GPU parallelization 하여 사용한다.
* additional trick:
        GPU communicates only in certain layers.
        layer 3에서 layer 4로 갈 때, 같은 GPU에 있는 kernel과만 연결한다.

### 3.3. Local Response Normalization

ReLU는 saturating을 방지할 필요가 없기 때문에, input normalization이 필수적이지 않다.

하지만, model의 generalization을 위해서는 normalization이 필요하다.

#### lateral Inhibition 
**측면 억제**라는 신경생리학 용어로, 한 신경 세포가 흥분했을 때, 주변의 신경세포 반응을 억제하는 현상이다.
이를 통해, 특별히 강한 신호를 내뿜는 뉴런(여기서는 특정 pixel?)이 있다면, 이를 주변과 '대비'시켜야 한다. 그 부분을 더 **잘  인식**하기 위해서다.

#### Why Local Response Normalization?
Local Response Normalizaiton layer는 lateral inhibition을 implement한다. 이는 특히 ReLU funciton을 사용할 때에 유용한데, 이는 upper bound가 없는 ReLU의 output을 normalizie 시킴과 동시에 특히 **강한 신호를 내뿜는 파트를 강하게 인식** 할 수 있기 때문이다. 또한 동시에, 강한 신호를 내뿜는다 하더라도 해당 지역이 전반적으로 강한 신호를 보인다면(특정 neuron만 강한 것이 아니라 locally 강한 경우) normalize를 통해 전반적인 신호의 세기를 줄일 수 있다.

#### Formulation

$$ b^i_{x, y} = a^i_{x, y}/(k + \alpha \sum_{j=max(0,i-n/2)}^{min(N-1, i+n/2} (a^j_{x,y})^2 )^\beta $$

where 
* $a^i_{x,y}$: activity of a neuron computed by appying kernel *i* at position $(x, y)$ and then applying the ReLU
* $b^i_{x,y}$: response-normalized activity

### 3.4. Overlapping Pooling

stride와 pooling kernel size를 같게 한다면 pooling이 overlap되지 않지만, stride의 크기를 줄임으로써 pooling을 overlap시킬 수 있다.

이를 통해 accuracy가 증가하였다.

---
## 4. Reducing Overfitting

### 4.1. Data Augmentation
overfitting을 방지하기 위한 가장 보편적인 방법은 label-preserving transformation을 통해 dataset의 크기를 늘리는 것이다.

본 paper에서는, 1개의 original image에서 여러개의 transformed images를 생성하기 위한 2가지 방법을 사용한다. 각각의 방법은 모두 연산량이 크지 않아 GPU가 특정 image batch를 training하는 동안, CPU에서 그 다음 batch에 대해 적용 할 수 있다.

#### Image Translation & Horiziontal Reflection
input 이미지는 256 x 256으로 조정이 된 상태인데, 여기서 다시 224x224 size의 patch들을 추출한다(그리고 각각을 y축 대칭시킨다). 이를 통해 각각의 image를 2048개의 image로 만들 수 있다(가로 32배, 세로 32배, 대칭으로 인해 2배). 물론 각각의 patch는 highly-interdependent이지만, overfitting을 효과적으로 방지 할 수 있다!(*왜 가능한 지는 이해하지 못했음*).

실제로 test에서 적용할 때에는, 4개의 corner patch와 1개의 central patch, 그리고 각각의 patch를 y축 대칭시켜 총 10개의 patch를 사용한다.

#### RGB channel의 intensity 조절
구체적으로는 PCA를 적용하는데, 각각의 RGB pixel에 $[p_1, p_2, p_3][\alpha_1 * \lambda_1, \alpha_2 * \lambda_2, \alpha_3 * \lambda_3]^T$ 를 더해준다.

*원리 이해 x!*

### 4.2. Dropout
